## Imports

In [20]:
from torch.utils.data import DataLoader
from torch.optim import Optimizer
from ultralytics import YOLO
import torch.nn as nn
import torch
import os
import timm
from ultralytics.nn import tasks
from ultralytics.nn import modules
import wandb
import matplotlib.pyplot as plt
import cv2
import random

In [21]:
#!pip install nbformat

In [22]:
#!pip install timm

# Setup and Configuration

In [23]:
'''
print(f"TIMM Version: {timm.__version__}")


try:
    # Point to the file created
    model = YOLO("yolov8_resnet50.yaml") 
    
    # Print the model info
    model.info()
    print("ResNet50 Backbone loaded successfully!")
    
except Exception as e:
    print(f"Something is wrong")'''

'\nprint(f"TIMM Version: {timm.__version__}")\n\n\ntry:\n    # Point to the file created\n    model = YOLO("yolov8_resnet50.yaml") \n\n    # Print the model info\n    model.info()\n    print("ResNet50 Backbone loaded successfully!")\n\nexcept Exception as e:\n    print(f"Something is wrong")'

In [ ]:
'''class TimmWrapper(nn.Module): #Wrapper Class that translates "timm" for YOLO
    def __init__(self, *args, **kwargs):
        super().__init__()
        #self.model = timm.create_model(name, pretrained=pretrained, features_only=True, out_indices=out_indices) 
        #self.channel_list = self.model.feature_info.channels() #Get the number of channels for output layer (YOLO head)
        model_name = "resnet50"
        if 'name' in kwargs:
             model_name = kwargs['name']
        elif len(args) > 2:
             model_name = args[2]
        
        is_pretrained = True
        config_dict = None

        if len(args) >3 and isinstance(args[3], dict):
             config_dict = args[3]
             is_pretrained = config_dict.get('pretrained', True)

        self.model = timm.create_model(model_name, pretrained=is_pretrained, feature_only=True, out_indices=[2, 3, 4])

        self.channel_list = [512, 1024, 2048] #Standard ResNet50 channels [256, 512, 1024, 2048]; we output P3, P4, P5 -> [512, 1024, 2048]

    def forward(self, x):
        return self.model(x)

class Index(nn.Module):#Takes list of P3,P4,P5 and picks one for head
        def __init__(self, *args, **kwargs):
            super().__init__()
            self.idx = 0
            try:
                if len(args) >= 3: self.idx = int(args[2])
                elif 'index' in kwargs: self.idx = int(kwargs['index'])
            except:
                 pass

        def forward(self,x):
            if isinstance(x, list) and len(x) > self.idx:
                return x[self.idx]
            return x



setattr(tasks, 'timm', TimmWrapper) #Inject wrapper into the Ultralytics internal registry
setattr(tasks, 'Index', Index) #avoid index out of range error, when yolo tries to access layer which are not available because of the changed backbone
tasks.timm = TimmWrapper
tasks.Index = Index
print("successfull")'''

'class TimmWrapper(nn.Module): #Wrapper Class that translates "timm" for YOLO\n    def __init__(self, *args, **kwargs):\n        super().__init__()\n        #self.model = timm.create_model(name, pretrained=pretrained, features_only=True, out_indices=out_indices) # Create the timm model (resnet50)\n        #self.channel_list = self.model.feature_info.channels() # Get the number of channels for each output layer (Crucial for YOLO head)\n        model_name = "resnet50"\n        if \'name\' in kwargs:\n             model_name = kwargs[\'name\']\n        elif len(args) > 2:\n             model_name = args[2]\n\n        is_pretrained = True\n        config_dict = None\n\n        if len(args) >3 and isinstance(args[3], dict):\n             config_dict = args[3]\n             is_pretrained = config_dict.get(\'pretrained\', True)\n\n        self.model = timm.create_model(model_name, pretrained=is_pretrained, feature_only=True, out_indices=[2, 3, 4])\n\n        self.channel_list = [512, 1024, 20

In [25]:
'''try:
    model = YOLO("yolov8_resnet50.yaml") 
    model.info()
    print("ResNet50 Backbone loaded successfully!")
except Exception as e:
    print(f"ERROR/n{e}")'''

'try:\n    model = YOLO("yolov8_resnet50.yaml") \n    model.info()\n    print("ResNet50 Backbone loaded successfully!")\nexcept Exception as e:\n    print(f"ERROR/n{e}")'

In [26]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }
parameters_dict = {
    'epochs': {
        'value': 50
        },
    'optimizer': {
        'value': 'sgd'
        },
    'lr0': {
        'value': 0.002
        },
    'batch': {
        'value': 16
        },
    }
sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [27]:
data_config = r"C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/dataset_final/data.yaml"
base_dir = r"C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection"
test_images_dir = os.path.join(base_dir, r"dataset_final/images/test")
#model_config = "yolov8_resnet50.yaml"

# Initialization

In [28]:
#print(f"Building model from: {model_config}")
#model = YOLO(model_config)

# Training Cell

## Weights & Biases

In [29]:
#manually initialize W&B
wandb.init( project="Bone_Fracture_Detection", name="ResNet_Fracture_Run_02", config={
        "architecture": "YOLOv8",
        "dataset": "GRAZPEDWRI-DX",
        "epochs": 50,
        "batch_size": 8,
        "augmentation": "Medical_v1"}
)

## Model training

In [30]:
model = YOLO("yolov8s.pt") # for default Yolov8 backbone yolov8_resnet50.yaml 


model.info()

YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [ ]:
def on_train_epoch_end(trainer):#Logging Callback
    epoch = trainer.epoch + 1
    metrics = trainer.metrics
    
    train_loss = trainer.loss_items[0].item() 
    map50 = metrics.get("metrics/mAP50(B)", 0)
    
    
    wandb.log({ 
        "Custom_Updates": map50,
        "Training_Loss": train_loss,
        "Epoch_Number": epoch
    })
    
    print(f"Epoch {epoch} | Loss: {train_loss:.4f} | mAP: {map50:.1%}")

model.add_callback("on_train_epoch_end", on_train_epoch_end)



## Actual Training

In [ ]:
print(f" Starting Run... Monitor at: {wandb.run.url}")

results = model.train(
    data=data_config, 
    imgsz=640, #increased because 640 might miss some hairline fractures, 800
    
    epochs=50,
    patience=15, #if no improvement after 15 epochs
    batch=, #8, 4
    optimizer = "SGD", #optimizer='AdamW',
    lr0= 0.005, #default of 0.01, reducing to prevent RuntimeError
    device=0,
    project="Bone_Fracture_Project",
    name="WandB_Run_YOLOv8s_class_balance_SGD_batch8_imgsz640",
    save=True, # to keep the best results
    #rect=True
    # Augmentation Settings
    degrees=10,      # Rotate +/- 10 degrees (Bones aren't always straight, but rarely upside down)
    translate=0.1,   # Shift image 10% (fracture might be off-center)
    scale=0.5,       # Zoom in/out (+/- 50%)
    fliplr=0.5,      # Flip Left-Right (Left hand looks like Right hand)
    flipud=0.0,      # NO flip Up-Down
    mosaic=1.0,      # Mix 4 images (Standard YOLO booster, very good for context)
    mixup=0.0,       # OFF: Do not mix two bones together (confusing for medical diagnosis)
    hsv_h=0.010,     # Color: Keep VERY low (X-rays are grayscale)
    hsv_s=0.0,       # Saturation: 0 (No color in X-rays)
    hsv_v=0.4,       # Brightness: +/- 30% (Simulates over/under-exposed X-rays) more Robust/Tougher than only 30%
)

# Uploads final artifacts and closes the connection
wandb.finish()

 Starting Run... Monitor at: https://wandb.ai/nils-pudenz-university-of-klagenfurt/Bone_Fracture_Detection/runs/al3q3xul
New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.250  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/dataset_final/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.0, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, li

Custom_Updates,▁▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
Epoch_Number,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training_Loss,▄▁▅▅▆▆▁▆▆▇▆▆▅▅▄▆▆▆▄▆▄▅▆▆▆▅▅▅▆█▄▇▅▄▄▄▄▃▂▁
Custom_Updates,0.71303
Epoch_Number,50
Training_Loss,0


In [33]:
'''model = YOLO(r"C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/notebooks/Bone_Fracture_Project/WandB_Run_YOLOv8s_class_balance3/weights/last.pt")
model.train(epoch=20,)'''

'model = YOLO(r"C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/notebooks/Bone_Fracture_Project/WandB_Run_YOLOv8s_class_balance3/weights/last.pt")\nmodel.train(epoch=20,)'

In [34]:
import os

PROJECT_ROOT = r"C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/notebooks/Bone_Fracture_Project"
os.chdir(PROJECT_ROOT)

print("CWD:", os.getcwd())

CWD: C:\Users\nilsp\OneDrive\Documents\GitHub\Bone_Fracture_Detection\notebooks\Bone_Fracture_Project


In [35]:
model = YOLO("C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/notebooks/Bone_Fracture_Project/WandB_Run_YOLOv8s_class_balance/weights/best.pt")
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
'''results = model.val(
    data=data_config, 
    conf=0.01
)'''

Ultralytics 8.3.250  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
Model summary (fused): 72 layers, 11,129,067 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 767.6660.8 MB/s, size: 515.3 KB)
val: Scanning C:\Users\nilsp\OneDrive\Documents\GitHub\Bone_Fracture_Detection\dataset_final\labels\val.cache... 2033 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2033/2033  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 128/128 2.9it/s 44.3s0.3s
                   all       2033       4856      0.702      0.676      0.687       0.43
           boneanomaly         19         34      0.507      0.272      0.282      0.142
              fracture       1402       1887        0.9      0.887      0.936       0.55
                 metal         70         79      0.927      0.975      0.976      0.775
    periostealreaction        234        354       0.66     

'results = model.val(\n    data=data_config, \n    conf=0.01\n)'

## Post Training - (Optional)

In [36]:
model.info()

Model summary (fused): 72 layers, 11,129,067 parameters, 0 gradients, 28.5 GFLOPs


(72, 11129067, 0, 28.4548608)

In [37]:
model_path = ''
model = YOLO(model_path)

print(f"Loading model from: {model_path}")
print(f"Predicting on: dataset_final(test)")

WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading model from: 
Predicting on: dataset_final(test)


In [ ]:
model.predict(
    source='C:/Users/nilsp/OneDrive/Documents/GitHub/Bone_Fracture_Detection/dataset_final/images/test',
    conf=0.5, #0.25
    save=True,
    augment=True,       
    save_txt=False,    
    save_conf=True,     
    stream=True
)

TypeError: model='' is not a supported model format. Ultralytics supports: ('PyTorch', 'TorchScript', 'ONNX', 'OpenVINO', 'TensorRT', 'CoreML', 'TensorFlow SavedModel', 'TensorFlow GraphDef', 'TensorFlow Lite', 'TensorFlow Edge TPU', 'TensorFlow.js', 'PaddlePaddle', 'MNN', 'NCNN', 'IMX', 'RKNN', 'ExecuTorch', 'Axelera')
See https://docs.ultralytics.com/modes/predict for help.

# References

In [ ]:
@software{yolov8_ultralytics,
  author = {Glenn Jocher and Ayush Chaurasia and Jing Qiu},
  title = {Ultralytics YOLOv8},
  version = {8.0.0},
  year = {2023},
  url = {https://github.com/ultralytics/ultralytics},
  orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
  license = {AGPL-3.0}
}